In [7]:
import cv2
import time
import subprocess
from collections import defaultdict
from ultralytics import YOLO

# Load YOLO model
model = YOLO('/Users/maximilian/Documents/Macbook Air 13/Studium/projects/wardrobe_detection_yolo/outputs/models/yolov8n_finetuned.pt')

# Correct label mapping
correct_names = [
    'adidas_samba',
    'adidas_spezial',
    'nike_killshot',
    'pants_dark',
    'pants_light',
    'shirt_lightblue',
    'shirt_lightlinen',
    'tshirt_dark',
    'tshirt_white'
]

# Initialize webcam
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

frames = []

print("Capturing 3 images with 0.3-seconds delay...")
for i in range(3):
    ret, frame = cap.read()
    if ret:
        frames.append(frame)
        print(f"Captured frame {i+1}")
    else:
        print(f"Failed to capture frame {i+1}")
    time.sleep(0.3)  # 0.3-seconds delay between captures

cap.release()

# Aggregate detections
label_stats = defaultdict(list)

print("Running YOLO inference on each frame...")
for frame in frames:
    results = model.predict(frame, show=False)[0]
    results.names = correct_names

    for box in results.boxes:
        class_id = int(box.cls[0])
        label = results.names[class_id]
        confidence = float(box.conf[0])
        label_stats[label].append(confidence)

# Compute average confidence
aggregated = {label: sum(confs) / len(confs) for label, confs in label_stats.items()}

# Build LLM prompt
prompt_context = "Objects detected across multiple frames:\n"
for label, avg_conf in aggregated.items():
    prompt_context += f"- {label} (avg confidence: {avg_conf:.2f})\n"

full_prompt = f"""
You are a wardrobe assistant. Based on the items detected, give outfit advice or a compliment.

{prompt_context}
"""

# Query Ollama via subprocess
def query_ollama(prompt, model='llama3.2:latest'):
    result = subprocess.run(
        ['ollama', 'run', model],
        input=prompt.encode('utf-8'),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode('utf-8')

# Run and display LLM output
print("Sending prompt to Ollama...")
response = query_ollama(full_prompt)
print("\nLLM Response:\n")
print(response)

Capturing 3 images with 0.3-seconds delay...
Captured frame 1
Captured frame 2
Captured frame 3
Running YOLO inference on each frame...

0: 384x640 (no detections), 35.9ms
Speed: 2.2ms preprocess, 35.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 adidas_samba, 31.1ms
Speed: 1.6ms preprocess, 31.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 adidas_samba, 29.8ms
Speed: 1.6ms preprocess, 29.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Sending prompt to Ollama...

LLM Response:

You look like you're going for a casual and comfortable vibe with that dark t-shirt! It's a great choice to pair with just about anything. I would suggest pairing it with some distressed denim jeans or black leggings for a relaxed, weekend look.

Alternatively, if you want to dress up the outfit, you could add a blazer or a cardigan to give it a more polished feel. And don't forget to add some sneakers or loafers to com

In [ ]:
import cv2
import time
import requests
import json
from collections import defaultdict
from ultralytics import YOLO

# Load YOLO model
model = YOLO('/Users/maximilian/Documents/Macbook Air 13/Studium/projects/wardrobe_detection_yolo/outputs/models/yolov8n_finetuned.pt')

# Correct label mapping
correct_names = [
    'adidas_samba',
    'adidas_spezial',
    'nike_killshot',
    'pants_dark',
    'pants_light',
    'shirt_lightblue',
    'shirt_lightlinen',
    'tshirt_dark',
    'tshirt_white'
]

# Initialize webcam
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

frames = []

print("Capturing 3 images with 0.3-second delay...")
for i in range(3):
    ret, frame = cap.read()
    if ret:
        frames.append(frame)
        print(f"Captured frame {i+1}")
    else:
        print(f"Failed to capture frame {i+1}")
    time.sleep(0.3)

cap.release()

# Aggregate detections
label_stats = defaultdict(list)

print("Running YOLO inference on each frame...")
for frame in frames:
    results = model.predict(frame, show=False)[0]
    results.names = correct_names

    for box in results.boxes:
        class_id = int(box.cls[0])
        label = results.names[class_id]
        confidence = float(box.conf[0])
        label_stats[label].append(confidence)

# Compute average confidence
aggregated = {label: sum(confs) / len(confs) for label, confs in label_stats.items()}

prompt_prefix = """You are a helpful wardrobe assistant.

Your job is to give supportive, casual outfit advice or compliments based on detected clothing items.

Here are some examples of how you might respond:
- "Nice combo! The white tee and light pants make for a clean summer look."
- "That's a great casual fit — especially those Adidas Spezial shoes."
- "Perfect for a relaxed day. You could throw on a jacket if it's chilly."

Now here's the input:
"""

# Build prompt
detected_context = "Objects detected across multiple frames:\n"
for label, avg_conf in aggregated.items():
    detected_context += f"- {label} (avg confidence: {avg_conf:.2f})\n"

full_prompt = prompt_prefix + detected_context

def stream_ollama(prompt, model='llama3.2'):
    url = 'http://localhost:11434/api/generate'
    headers = {'Content-Type': 'application/json'}
    payload = {
        'model': model,
        'prompt': prompt,
        'stream': True
    }

    print("\nLLM Response:\n")
    response = requests.post(url, json=payload, stream=True)
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8'))
                print(data.get("response", ""), end='', flush=True)
            except json.JSONDecodeError:
                continue  # skip bad lines silently

# Run streaming query
stream_ollama(full_prompt)

2025-06-03 23:48:22.262 Python[80137:967805] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Capturing 3 images with 0.3-second delay...
Captured frame 1
Captured frame 2
Captured frame 3
Running YOLO inference on each frame...

0: 384x640 (no detections), 32.6ms
Speed: 2.0ms preprocess, 32.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 adidas_samba, 33.4ms
Speed: 1.6ms preprocess, 33.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 adidas_samba, 33.0ms
Speed: 1.5ms preprocess, 33.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

LLM Response:

You have a versatile and comfortable base layer with your dark T-shirt! I'd recommend pairing it with some versatile bottoms to create a stylish outfit.

How about trying a pair of distressed denim jeans or a flowy black skirt to add some visual interest? You could also consider adding a statement jacket, like a leather biker jacket or a denim jacket with embroidery, to give your outfit a cool edge.

If you want to keep things simple, you could sti

In [4]:
import cv2
import time
import json
import requests
from collections import defaultdict
from ultralytics import YOLO

# ------------------ CONFIG ------------------ #

# YOLO model path and label map
model = YOLO('/Users/maximilian/Documents/Macbook Air 13/Studium/projects/wardrobe_detection_yolo/outputs/models/yolov8n_finetuned.pt')
correct_names = [
    'adidas_samba', 'adidas_spezial', 'nike_killshot',
    'pants_dark', 'pants_light',
    'shirt_lightblue', 'shirt_lightlinen',
    'tshirt_dark', 'tshirt_white'
]

# Prompt prefix (constant)
prompt_prefix = """You are a wardrobe assistant. Give a friendly compliment or fashion tip based on the detected outfit and user request.

Example:
Input:
- tshirt_white (confidence: 0.91)
- pants_light (confidence: 0.88)
User: What should I wear if I go out tonight?

Output:
That's a clean combo. You could throw on a bomber jacket or denim to add some personality.

---

Now your turn:
"""

# ------------------ FUNCTIONS ------------------ #

def capture_frames(n=3, delay=0.):
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        raise RuntimeError("Could not open webcam.")
    frames = []
    for i in range(n):
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        time.sleep(delay)
    cap.release()
    return frames

def detect_clothing(frames):
    label_stats = defaultdict(list)
    for frame in frames:
        results = model.predict(frame, show=False)[0]
        results.names = correct_names
        for box in results.boxes:
            class_id = int(box.cls[0])
            label = results.names[class_id]
            confidence = float(box.conf[0])
            label_stats[label].append(confidence)
    aggregated = {label: sum(c) / len(c) for label, c in label_stats.items()}
    return aggregated

def build_prompt(aggregated, user_input):
    detected = '\n'.join(f"- {label} (confidence: {conf:.2f})" for label, conf in aggregated.items())
    return f"{prompt_prefix}Input:\n{detected}\nUser: {user_input}\nOutput:"

def query_ollama_stream(prompt, model='llama3.2'):
    url = 'http://localhost:11434/api/generate'
    payload = {'model': model, 'prompt': prompt, 'stream': True}
    print("\nLLM Response:\n")
    response = requests.post(url, json=payload, stream=True)
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8'))
                print(data.get("response", ""), end='', flush=True)
            except json.JSONDecodeError:
                continue

# ------------------ INTERACTION ------------------ #

# Round 1: Detect + Prompt
print("Step 1: Capturing your outfit...")
frames = capture_frames()
aggregated = detect_clothing(frames)

user_input = input("\nWhat do you want to ask or say to the mirror assistant? (e.g., 'Is this good for a date?')\n> ")
full_prompt = build_prompt(aggregated, user_input)
query_ollama_stream(full_prompt)

# Round 2: New detection + Prompt
print("\n\nStep 2: Capturing updated outfit (e.g. changed something)...")
frames = capture_frames()
aggregated = detect_clothing(frames)

user_input = input("\nAnything else you want to ask now?\n> ")
full_prompt = build_prompt(aggregated, user_input)
query_ollama_stream(full_prompt)

Step 1: Capturing your outfit...

0: 384x640 (no detections), 33.8ms
Speed: 1.3ms preprocess, 33.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 34.3ms
Speed: 1.5ms preprocess, 34.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 adidas_samba, 31.3ms
Speed: 1.4ms preprocess, 31.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

LLM Response:

Input:
- tshirt_dark (confidence: 0.51)

User: How do I look?

Output:
Honestly, it's a bit of an understated look for you. Why not add some depth with a statement piece like a patterned scarf or a bold necklace to elevate your dark tee?

Step 2: Capturing updated outfit (e.g. changed something)...

0: 384x640 (no detections), 34.1ms
Speed: 1.3ms preprocess, 34.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 32.4ms
Speed: 1.5ms preprocess, 32.4ms inference, 0.3ms postprocess per image at shape (1, 